In [1]:
import numpy as np
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import math
import scipy.linalg as la
from scipy import optimize as opt
from LinApp_FindSS import LinApp_FindSS
from LinApp_Deriv import LinApp_Deriv
from LinApp_Solve import LinApp_Solve

In [ ]:
alpha = 0.33
k = 5.0
z = 1.0
b = 2.0
t =0.1
h = 24.0

def equation1(inputs, params):
    r, k, c, T = inputs
    gamma, ci, beta, alpha, a, delta, z, tao, l = params
    
    nd = k * (((1 - alpha) * z) / w) ** (1/alpha)
    ns = h - (b / (w * (1 + b))) * (w * h + )
    
    diff1 = c - (1 - tao) * ((r - delta) * k) - T
    diff2 = 1 - beta * ((r - delta) * (1- tao) + 1)
    diff4 = r - alpha * (k ** (alpha - 1)) * ((l * np.exp(z)) ** (1 - alpha))
    diff6 = tao * (r - delta) * k  - T
    diff_vec = np.array([diff1, diff2, diff4, diff6])
    return diff_vec

r_init = 0.1
k_init = 5
c_init = 0.1
T_init = 0.1

init_vec = np.array([r_init, k_init, c_init, T_init])
params = [gamma, ci, beta, alpha, a, delta, z, tao, l]
#bounds = ((1e-10, None), (1e-10, None))
#results = opt.minimize(equations1, init_vec, args = params, method = 'L-BFGS-B',bounds = bounds, tol = 1e-10)
results2 = opt.fsolve(equations11, init_vec, args = params)

print("Numerical Method: r =", results2[0], ", K = ",results2[1], ", c = ", results2[2], ", T = ",results2[3])
print("These match up with our algebraic solutions.")